# PIP-PRoS: Controlo do braço robótico

Neste notebook serão introduzidos os métodos usados para o controlo do braço robótico. Todos os métodos aqui apresentados fazem parte da biblioteca pyniryo2, cuja documentação pode ser acessada por [aqui](https://docs.niryo.com/dev/pyniryo2/v1.0.0/en/index.html).

Para mais informações, acesse [o site do projeto](https://sites.google.com/tecnico.ulisboa.pt/pip-pros/)!

## Bibliotecas utilizadas

Como comentado, a única biblioteca necessária para controlar o braço robótico é a ```PyNiryo2```, que deve ser importada como apresentado abaixo. 

A biblioteca ```time``` será usada para introduzir delays após a chamada de certas funções.

In [23]:
from pyniryo2 import *
import time

## Configuração do braço robótico

A primeira etapa para o uso do robot é abrir a conexão e instanciar um objeto da classe NiryoRobot. Existem várias formas de conectar o braço, por simplicidade abaixo usamos a conexão via Ethernet, mas também é possível conectar o braço na mesma rede do computador que será usado para o controle via Wi-Fi.

In [ ]:
robot = NiryoRobot("169.254.200.200")

Em seguida é necessário realizar a calibração do braço. Este método deve ser chamado sempre após a conexão com o braço robótico.

In [ ]:
robot.arm.calibrate_auto()

O próximo passo é atribuir a velocidade de operação do braço. Isso pode ser feito por meio do método ```robot.arm.set_arm_max_velocity(percentage_speed)```. No entanto na implementação atual da biblioteca não são permitidos valores para ```percentage_speed``` que ultrapassam 100, o que torna necessária uma implementação alternativa deste método como a encontrada abaixo.

In [ ]:
def custom_set_arm_max_velocity(arm, percentage_speed):
    arm._check_range_belonging(percentage_speed, 1, 200)
    req = arm._services.get_max_velocity_scaling_factor_request(percentage_speed)
    resp = arm._services.set_max_velocity_scaling_factor_service.call(req)
    arm._check_result_status(resp)

custom_set_arm_max_velocity(robot.arm, 200)

## Movimentação do braço robótico

Os dois métodos principais usados para realizar o controle de posição do braço robótico são: ```robot.arm.stop_move()``` e ```robot.arm.move_linear_pose(pose)```. 

O primeiro é responsável por interromper o movimento atual do braço, e é usado antes de trocar o setpoint que o braço segue enquanto ele ainda se move. Para que o método funcione é importante incluir um ```time.sleep(0.01)``` após sua chamada.

In [ ]:
robot.arm.stop_move()
time.sleep(0.01)

O segundo método é responsável por mudar, por meio de um movimento linear, a pose atual do robô definida por um vetor [x, y, z, roll, pitch, yaw].

Devido a um erro na biblioteca da Niryo, não é possível usar esse método de forma _non-blocking_, ou seja, caso seja chamado o programa espera que o movimento seja finalizado para passar para a próxima linha. Isso é um problema para o programa como foi concebido, já que idealmente devemos contar com uma captura contínua de frames e interromper o programa durante o movimento do braço impediria que isso acontecesse.

Para solucionar esse problema, assim como no caso do ```robot.arm.set_arm_max_velocity```, foi criada uma implementação alternativa do método, como vista abaixo:

In [ ]:
def non_blocking_move_linear_position(arm, pose, callback=None):
    arm._actions.get_move_linear_pose_goal(pose).send(result_callback=callback)

Dessa forma é possível movimentar o braço de forma ```non-blocking``` até uma pose conhecida, como abaixo:

In [ ]:
aruco_0_pose = np.array([0.14, 0.28, 0.09, 0.0, 1.57, 0.0])
non_blocking_move_linear_position(robot.arm, aruco_0_pose)